In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
import scipy.stats as stats
from scipy.stats import ttest_rel

Read in files and perform statistical tests of independence for Pearson scores at each walk length

In [8]:
# Read data
main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v2/dataset/'
# main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v2/scrambled_dataset/'
file_list = Path(main_path).glob('*batch??.csv')
for file in file_list:
    print(file)
    df = pd.read_csv(file)
    # ,ave to a copy of df, then perform Fisher's r to z transformation on Pearson scores
    df_z = df.copy()
    for i in range(1, 6):
        df_z[str(i)] = np.arctanh(df_z[str(i)])
    # print(df_z)
    print(file.stem)
    # compare difference in score between Recon methods for each walk length column
    for i in range(1, 6):
        print('Walk Length: '+str(i))
        # get unique values of Recon column
        recon_list = df['Recon'].unique()
        df_stats_all = pd.DataFrame()
        df_stats = pd.DataFrame()
        # loop through each unique value of Recon column
        for recon in recon_list:
            print(recon)
            print(df[df['Recon'] == str(recon)][str(i)])
            print('Statistics for '+str(recon))
            print('Mean')
            print(df[df['Recon'] == str(recon)][str(i)].mean())
            print('Stdev')
            print(df[df['Recon'] == str(recon)][str(i)].std())
            print('Median')
            print(df[df['Recon'] == str(recon)][str(i)].median())
            print('IQR')
            print(df[df['Recon'] == str(recon)][str(i)].quantile(
                q=0.75)-df[df['Recon'] == str(recon)][str(i)].quantile(q=0.25))
            # combine statistics from each Recon method into dataframe
            df_stats = pd.DataFrame({'Recon': recon_list, 'Mean': df[df['Recon'] == str(recon)][str(i)].mean(), 'Stdev': df[df['Recon'] == str(recon)][str(i)].std(
            ), 'Median': df[df['Recon'] == str(recon)][str(i)].median(), 'IQR': df[df['Recon'] == str(recon)][str(i)].quantile(q=0.75)-df[df['Recon'] == str(recon)][str(i)].quantile(q=0.25)})
            # combine df_stats from each Recon method into dataframe
            df_stats_all = pd.concat([df_stats, df_stats_all], axis=0)
        print(df_stats_all)
        # save dataframe to csv
        df_stats_all.to_csv(main_path+'/stats/'+file.stem +
                            '_walk_length_'+str(i)+'_stats.csv', index=False)
        # calculate p-value for difference in score between Recon methods in recon_list, with comparisons between each pair of Recon methods, save all statistics and p-values to csv
        df_ttest_results = pd.DataFrame()
        df_ttest_rel_results = pd.DataFrame()
        df_z_ttest_results = pd.DataFrame()
        df_z_ttest_rel_results = pd.DataFrame()
        for recon in recon_list:
            for recon2 in recon_list:
                if recon != recon2:
                    print(recon)
                    print(recon2)
                    # print the length of df for each Recon method
                    print(len(df[df['Recon'] == str(recon)][str(i)]))
                    print(len(df[df['Recon'] == str(recon2)][str(i)]))
                    # calculate t-test on Pearson scores, get t-statistic and p-value
                    ttest_pearson_score = stats.ttest_ind(df[df['Recon'] == str(
                        recon)][str(i)], df[df['Recon'] == str(recon2)][str(i)], alternative='less', nan_policy='propagate')
                    ttest_rel_pearson_score = ttest_rel(df[df['Recon'] == str(
                        recon)][str(i)], df[df['Recon'] == str(recon2)][str(i)], alternative='less', nan_policy='propagate')
                    ttest_z_score = stats.ttest_ind(df_z[df_z['Recon'] == str(
                        recon)][str(i)], df_z[df_z['Recon'] == str(recon2)][str(i)], alternative='less', nan_policy='propagate')
                    ttest_rel_z_score = ttest_rel(df_z[df_z['Recon'] == str(
                        recon)][str(i)], df_z[df_z['Recon'] == str(recon2)][str(i)], alternative='less', nan_policy='propagate')
                    # combine t-test results into dataframe
                    df_ttest_results = pd.concat([df_ttest_results, pd.DataFrame({'Recon 1': recon, 'Recon 2': recon2, 'T-statistic Pearson Score t-test': ttest_pearson_score[0],
                                                 'p-value Pearson Score t-test':ttest_pearson_score[1]}, index=[0])], axis=0)  # ,'T-statistic z-score':ttest_z_score[0],'p-value z-score':ttest_z_score[1]
                    df_ttest_rel_results = pd.concat([df_ttest_rel_results, pd.DataFrame({'Recon 1': recon, 'Recon 2': recon2, 'T-statistic Pearson Score t-test': ttest_rel_pearson_score[0],
                                                                                          'p-value Pearson Score t-test':ttest_rel_pearson_score[1]}, index=[0])], axis=0)  # ,'T-statistic z-score':ttest_z_score[0],'p-value z-score':ttest_z_score[1]
                    df_z_ttest_rel_results = pd.concat([df_z_ttest_rel_results, pd.DataFrame({'Recon 1': recon, 'Recon 2': recon2, 'T-statistic z-score t-test': ttest_rel_z_score[0],
                                                                                              'p-value z-score t-test':ttest_rel_z_score[1]}, index=[0])], axis=0)  # ,'T-statistic z-score':ttest_z_score[0],'p-value z-score':ttest_z_score[1]
                    df_z_ttest_results = pd.concat([df_z_ttest_results, pd.DataFrame({'Recon 1': recon, 'Recon 2': recon2, 'T-statistic z-score t-test': ttest_z_score[0],
                                                    'p-value z-score t-test':ttest_z_score[1]}, index=[0])], axis=0)  # ,'T-statistic z-score':ttest_z_score[0],'p-value z-score':ttest_z_score[1]
        print('T-test ind results')
        print(df_ttest_results)
        print('T-test rel results')
        print(df_ttest_rel_results)
        print('T-test ind z-score results')
        print(df_z_ttest_results)
        print('T-test rel z-score results')
        print(df_z_ttest_rel_results)

        # save dataframe to csv
        df_ttest_results.to_csv(
            main_path+'/stats/'+file.stem+'_walk_length_'+str(i)+'_ttest_results.csv', index=False)
        df_ttest_rel_results.to_csv(
            main_path+'/stats/'+file.stem+'_walk_length_'+str(i)+'_ttest_rel_results.csv', index=False)
        df_z_ttest_results.to_csv(
            main_path+'/stats/'+file.stem+'_walk_length_'+str(i)+'_z_score_ttest_results.csv', index=False)
        df_z_ttest_rel_results.to_csv(
            main_path+'/stats/'+file.stem+'_walk_length_'+str(i)+'_z_score_ttest_rel_results.csv', index=False)
        # # calculate non-parametric statistics for difference in score between Recon methods in recon_list, with comparisons between each pair of Recon methods, save all statistics and p-values to csv
        # df_mannwhitneyu_results = pd.DataFrame()
        # for recon in recon_list:
        #     for recon2 in recon_list:
        #         if recon != recon2:
        #             # print(recon)
        #             # print(recon2)
        #             # print(stats.mannwhitneyu(df[df['Recon']==str(recon)][str(i)],df[df['Recon']==str(recon2)][str(i)]))
        #             # calculate Mann-Whitney U test on Pearson scores, get U-statistic and p-value
        #             mannwhitneyu = stats.mannwhitneyu(df[df['Recon'] == str(
        #                 recon)][str(i)], df[df['Recon'] == str(recon2)][str(i)])
        #             # # calculate Mann-Whitney U test on Fisher's r to z transformed Pearson scores, get U-statistic and p-value
        #             # mannwhitneyu_z = stats.mannwhitneyu(df_z[df_z['Recon']==str(recon)]['z'],df_z[df_z['Recon']==str(recon2)]['z'])
        #             # combine Mann-Whitney U test results into dataframe
        #             # , 'U-statistic z-score':mannwhitneyu_z[0],'p-value z-score':mannwhitneyu_z[1]
        #             df_mannwhitneyu_results = pd.concat([df_mannwhitneyu_results, pd.DataFrame(
        #                 {'Recon 1': recon, 'Recon 2': recon2, 'U-statistic Pearson Score Mann-Whitney': mannwhitneyu[0], 'p-value Pearson Score Mann-Whitney':mannwhitneyu[1]}, index=[0])], axis=0)
        # # save dataframe to csv
        # df_mannwhitneyu_results.to_csv(
        #     main_path+'/'+file.stem+'_walk_length_'+str(i)+'_mannwhitneyu_results.csv', index=False)

/home/paul/thesis/dev/SAY_sf_prediction_v2/dataset/count_all_batch41.csv
count_all_batch41
Walk Length: 1
MSMT CSD SIFT2 Streamline Count
0     0.613709
1     0.615947
2     0.459425
3     0.537682
4     0.661649
5     0.422555
6     0.603934
7     0.622703
8     0.634884
9     0.628968
10    0.575197
11    0.610931
12    0.640958
13    0.563378
14    0.621856
15    0.572169
16    0.596888
17    0.591063
18    0.558532
19    0.494764
20    0.532592
Name: 1, dtype: float64
Statistics for MSMT CSD SIFT2 Streamline Count
Mean
0.579037373184194
Stdev
0.06130506207767524
Median
0.5968877830015069
IQR
0.06332347969518848
GQI Streamline Count
21    0.593448
22    0.592647
23    0.454313
24    0.546210
25    0.644447
26    0.430576
27    0.556105
28    0.615597
29    0.608029
30    0.585431
31    0.540963
32    0.594848
33    0.609132
34    0.565851
35    0.592926
36    0.558713
37    0.562922
38    0.570820
39    0.531897
40    0.478631
41    0.516424
Name: 1, dtype: float64
Statistics for GQ

/home/paul/.local/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1956: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0
/home/paul/.local/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1956: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


GQI Streamline Count
DTI Streamline Count
21
21
DTI Streamline Count
MSMT CSD SIFT2 Streamline Count
21
21
DTI Streamline Count
GQI Streamline Count
21
21
T-test ind results
                           Recon 1                          Recon 2  \
0  MSMT CSD SIFT2 Streamline Count             GQI Streamline Count   
0  MSMT CSD SIFT2 Streamline Count             DTI Streamline Count   
0             GQI Streamline Count  MSMT CSD SIFT2 Streamline Count   
0             GQI Streamline Count             DTI Streamline Count   
0             DTI Streamline Count  MSMT CSD SIFT2 Streamline Count   
0             DTI Streamline Count             GQI Streamline Count   

   T-statistic Pearson Score t-test  p-value Pearson Score t-test  
0                               NaN                           NaN  
0                               NaN                           NaN  
0                               NaN                           NaN  
0                               NaN                     

/home/paul/.local/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1956: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0
/home/paul/.local/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1956: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0



MSMT CSD SIFT2 Streamline Count
21
21
GQI Streamline Count
DTI Streamline Count
21
21
DTI Streamline Count
MSMT CSD SIFT2 Streamline Count
21
21
DTI Streamline Count
GQI Streamline Count
21
21
T-test ind results
                           Recon 1                          Recon 2  \
0  MSMT CSD SIFT2 Streamline Count             GQI Streamline Count   
0  MSMT CSD SIFT2 Streamline Count             DTI Streamline Count   
0             GQI Streamline Count  MSMT CSD SIFT2 Streamline Count   
0             GQI Streamline Count             DTI Streamline Count   
0             DTI Streamline Count  MSMT CSD SIFT2 Streamline Count   
0             DTI Streamline Count             GQI Streamline Count   

   T-statistic Pearson Score t-test  p-value Pearson Score t-test  
0                               NaN                           NaN  
0                               NaN                           NaN  
0                               NaN                           NaN  
0                

...

T-test ind results
                           Recon 1                          Recon 2  \
0  MSMT CSD SIFT2 Streamline Count             GQI Streamline Count   
0  MSMT CSD SIFT2 Streamline Count             DTI Streamline Count   
0             GQI Streamline Count  MSMT CSD SIFT2 Streamline Count   
0             GQI Streamline Count             DTI Streamline Count   
0             DTI Streamline Count  MSMT CSD SIFT2 Streamline Count   
0             DTI Streamline Count             GQI Streamline Count   

   T-statistic Pearson Score t-test  p-value Pearson Score t-test  
0                               NaN                           NaN  
0                               NaN                           NaN  
0                               NaN                           NaN  
0                               NaN                           NaN  
0                               NaN                           NaN  
0                               NaN                           NaN  
T-test 

/home/paul/.local/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1956: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0
/home/paul/.local/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1956: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0
/home/paul/.local/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1956: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


                  Recon      Mean     Stdev    Median       IQR
0  MSMT CSD Mean Length  0.542501  0.068310  0.535020  0.110485
1       GQI Mean Length  0.542501  0.068310  0.535020  0.110485
2       DTI Mean Length  0.542501  0.068310  0.535020  0.110485
0  MSMT CSD Mean Length  0.517675  0.063036  0.522445  0.076611
1       GQI Mean Length  0.517675  0.063036  0.522445  0.076611
2       DTI Mean Length  0.517675  0.063036  0.522445  0.076611
0  MSMT CSD Mean Length  0.456080  0.074864  0.467311  0.096309
1       GQI Mean Length  0.456080  0.074864  0.467311  0.096309
2       DTI Mean Length  0.456080  0.074864  0.467311  0.096309
MSMT CSD Mean Length
GQI Mean Length
21
21
MSMT CSD Mean Length
DTI Mean Length
21
21
GQI Mean Length
MSMT CSD Mean Length
21
21
GQI Mean Length
DTI Mean Length
21
21
DTI Mean Length
MSMT CSD Mean Length
21
21
DTI Mean Length
GQI Mean Length
21
21
T-test ind results
                Recon 1               Recon 2  \
0  MSMT CSD Mean Length       GQI Mean Lengt

/home/paul/.local/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1956: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0
/home/paul/.local/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1956: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


                  Recon      Mean     Stdev    Median       IQR
0  MSMT CSD Mean Length  0.551084  0.070191  0.535277  0.118328
1       GQI Mean Length  0.551084  0.070191  0.535277  0.118328
2       DTI Mean Length  0.551084  0.070191  0.535277  0.118328
0  MSMT CSD Mean Length  0.515405  0.059612  0.515827  0.064427
1       GQI Mean Length  0.515405  0.059612  0.515827  0.064427
2       DTI Mean Length  0.515405  0.059612  0.515827  0.064427
0  MSMT CSD Mean Length  0.500395  0.067947  0.511294  0.081773
1       GQI Mean Length  0.500395  0.067947  0.511294  0.081773
2       DTI Mean Length  0.500395  0.067947  0.511294  0.081773
MSMT CSD Mean Length
GQI Mean Length
21
21
MSMT CSD Mean Length
DTI Mean Length
21
21
GQI Mean Length
MSMT CSD Mean Length
21
21
GQI Mean Length
DTI Mean Length
21
21
DTI Mean Length
MSMT CSD Mean Length
21
21
DTI Mean Length
GQI Mean Length
21
21
T-test ind results
                Recon 1               Recon 2  \
0  MSMT CSD Mean Length       GQI Mean Lengt

Make Box Plots

In [9]:
# Read data
main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v2/dataset/'
# main_path = '/home/paul/thesis/dev/SAY_sf_prediction_v3/scrambled_dataset/'
file_list = Path(main_path).glob('*batch??.csv')
for file in file_list:
    print(file)
    df = pd.read_csv(file)
    print(file.stem)
    # melt data for boxplot
    dd=pd.melt(df,id_vars=['Recon'],value_vars=['1','2','3','4','5'],var_name='Walk Length')
    # seaborn boxplot with hue based on recon method
    sns.boxplot(x='Walk Length',y='value',data=dd,hue='Recon')
    # plt.show()
    plt.ylabel('Pearson Score')
    # save figure
    plt.savefig(main_path+'/figures/'+file.stem+'_box_plot.png')
    plt.close()

/home/paul/thesis/dev/SAY_sf_prediction_v2/dataset/count_all_batch41.csv
count_all_batch41
/home/paul/thesis/dev/SAY_sf_prediction_v2/dataset/volume_weighted_all_batch42.csv
volume_weighted_all_batch42
/home/paul/thesis/dev/SAY_sf_prediction_v2/dataset/count_all_batch46.csv
count_all_batch46
/home/paul/thesis/dev/SAY_sf_prediction_v2/dataset/volume_weighted_all_batch46.csv
volume_weighted_all_batch46
/home/paul/thesis/dev/SAY_sf_prediction_v2/dataset/count_all_batch32.csv
count_all_batch32
/home/paul/thesis/dev/SAY_sf_prediction_v2/dataset/mean_length_all_batch42.csv
mean_length_all_batch42
/home/paul/thesis/dev/SAY_sf_prediction_v2/dataset/volume_weighted_all_batch31.csv
volume_weighted_all_batch31
/home/paul/thesis/dev/SAY_sf_prediction_v2/dataset/mean_length_all_batch46.csv
mean_length_all_batch46
/home/paul/thesis/dev/SAY_sf_prediction_v2/dataset/count_all_batch33.csv
count_all_batch33
/home/paul/thesis/dev/SAY_sf_prediction_v2/dataset/mean_length_all_batch47.csv
mean_length_all_ba